In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1717205,2021-10-04,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1717206,2021-10-05,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1717207,2021-10-06,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1717208,2021-10-07,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
44799,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
44801,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
44803,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
44805,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
44807,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1046382,2021-10-04,Arkansas,Arkansas,5001,3400,56.00,5000,Arkansas,AR,Arkansas,State,3017804
1046384,2021-10-05,Arkansas,Arkansas,5001,3414,56.00,5000,Arkansas,AR,Arkansas,State,3017804
1046386,2021-10-06,Arkansas,Arkansas,5001,3417,56.00,5000,Arkansas,AR,Arkansas,State,3017804
1046388,2021-10-07,Arkansas,Arkansas,5001,3424,56.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1717205,2021-10-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1717206,2021-10-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1717207,2021-10-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1717208,2021-10-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-10-08') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
626,2021-10-08,Snohomish,Washington,60795,714.00,53061,WA,County,822083,7395.24,86.85
1250,2021-10-08,Cook,Illinois,626475,11664.00,17031,IL,County,5150233,12164.01,226.48
1873,2021-10-08,Orange,California,321147,5475.00,6059,CA,County,3175692,10112.66,172.40
2495,2021-10-08,Maricopa,Arizona,707687,11670.00,4013,AZ,County,4485414,15777.52,260.18
3117,2021-10-08,Los Angeles,California,1469338,26284.00,6037,CA,County,10039107,14636.14,261.82
...,...,...,...,...,...,...,...,...,...,...,...
1715889,2021-10-08,Wheeler,Oregon,83,1.00,41069,OR,County,1332,6231.23,75.08
1716250,2021-10-08,King,Texas,15,0.00,48269,TX,County,272,5514.71,0.00
1716580,2021-10-08,Esmeralda,Nevada,58,1.00,32009,NV,County,873,6643.76,114.55
1716906,2021-10-08,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
626,2021-10-08,Snohomish,Washington,60795,714.00,53061,WA,County,822083,7395.24,86.85,86.85,7395.24
1250,2021-10-08,Cook,Illinois,626475,11664.00,17031,IL,County,5150233,12164.01,226.48,226.48,12164.01
1873,2021-10-08,Orange,California,321147,5475.00,6059,CA,County,3175692,10112.66,172.40,172.40,10112.66
2495,2021-10-08,Maricopa,Arizona,707687,11670.00,4013,AZ,County,4485414,15777.52,260.18,260.18,15777.52
3117,2021-10-08,Los Angeles,California,1469338,26284.00,6037,CA,County,10039107,14636.14,261.82,261.82,14636.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1715889,2021-10-08,Wheeler,Oregon,83,1.00,41069,OR,County,1332,6231.23,75.08,75.08,6231.23
1716250,2021-10-08,King,Texas,15,0.00,48269,TX,County,272,5514.71,0.00,0.00,5514.71
1716580,2021-10-08,Esmeralda,Nevada,58,1.00,32009,NV,County,873,6643.76,114.55,114.55,6643.76
1716906,2021-10-08,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00,0.00,4733.73


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1094461,2021-10-08,Hale,Alabama,2960,86.00,1065,AL,County,14651,20203.40,586.99,586.99,20203.40,1
1192031,2021-10-08,Clarke,Alabama,4710,80.00,1025,AL,County,23622,19939.04,338.67,338.67,19939.04,2
309212,2021-10-08,Calhoun,Alabama,21924,444.00,1015,AL,County,113605,19298.45,390.83,390.83,19298.45,3
757688,2021-10-08,Etowah,Alabama,19652,486.00,1055,AL,County,102268,19216.18,475.22,475.22,19216.18,4
635226,2021-10-08,Franklin,Alabama,5904,103.00,1059,AL,County,31362,18825.33,328.42,328.42,18825.33,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631274,2021-10-08,Weston,Wyoming,947,8.00,56045,WY,County,6927,13671.14,115.49,115.49,13671.14,19
920835,2021-10-08,Johnson,Wyoming,1119,15.00,56019,WY,County,8445,13250.44,177.62,177.62,13250.44,20
1044713,2021-10-08,Sublette,Wyoming,1266,13.00,56035,WY,County,9831,12877.63,132.23,132.23,12877.63,21
1300258,2021-10-08,Lincoln,Wyoming,2515,15.00,56023,WY,County,19830,12682.80,75.64,75.64,12682.80,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
855081,2021-10-08,Lowndes,Alabama,1725,61.00,1085,AL,County,9726,17735.97,627.18,627.18,17735.97,17,1
1094461,2021-10-08,Hale,Alabama,2960,86.00,1065,AL,County,14651,20203.40,586.99,586.99,20203.40,1,2
362219,2021-10-08,Walker,Alabama,10955,350.00,1127,AL,County,63521,17246.27,551.00,551.00,17246.27,27,3
853957,2021-10-08,Crenshaw,Alabama,2418,75.00,1041,AL,County,13772,17557.36,544.58,544.58,17557.36,20,4
922518,2021-10-08,Greene,Alabama,1240,44.00,1063,AL,County,8111,15287.88,542.47,542.47,15287.88,48,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267572,2021-10-08,Uinta,Wyoming,3660,26.00,56041,WY,County,20226,18095.52,128.55,128.55,18095.52,5,19
1631274,2021-10-08,Weston,Wyoming,947,8.00,56045,WY,County,6927,13671.14,115.49,115.49,13671.14,19,20
1300258,2021-10-08,Lincoln,Wyoming,2515,15.00,56023,WY,County,19830,12682.80,75.64,75.64,12682.80,22,21
851709,2021-10-08,Albany,Wyoming,5947,29.00,56001,WY,County,38880,15295.78,74.59,74.59,15295.78,13,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,86.85,7395.24,27,29
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,86.85,7395.24,27,29
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,86.85,7395.24,27,29
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,86.85,7395.24,27,29
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,86.85,7395.24,27,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1713222,2021-10-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1713223,2021-10-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1713224,2021-10-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1713225,2021-10-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1089921,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,586.99,20203.40,2,1,1.00,nan
1089922,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,586.99,20203.40,2,1,0.00,nan
1089923,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,586.99,20203.40,2,1,0.00,nan
1089924,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,586.99,20203.40,2,1,0.00,nan
1089925,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,586.99,20203.40,2,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1322262,2021-10-04,Crook,Wyoming,792,17.00,56011,WY,County,7584,10443.04,224.16,224.16,10759.49,7,23,12.00,6.14
1322263,2021-10-05,Crook,Wyoming,799,17.00,56011,WY,County,7584,10535.34,224.16,224.16,10759.49,7,23,7.00,5.79
1322264,2021-10-06,Crook,Wyoming,803,17.00,56011,WY,County,7584,10588.08,224.16,224.16,10759.49,7,23,4.00,5.71
1322265,2021-10-07,Crook,Wyoming,805,17.00,56011,WY,County,7584,10614.45,224.16,224.16,10759.49,7,23,2.00,5.14


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa[ '14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
851657,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,627.18,17735.97,1,17,1.00,13.07,0.00,nan
851658,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,627.18,17735.97,1,17,0.00,11.64,0.00,nan
851659,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,627.18,17735.97,1,17,0.00,10.64,0.00,nan
851660,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,627.18,17735.97,1,17,0.00,9.36,0.00,nan
851661,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,627.18,17735.97,1,17,0.00,8.29,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358208,2021-10-04,Teton,Wyoming,4953,13.00,56039,WY,County,23464,21108.93,55.40,55.40,21241.05,23,1,10.00,10.64,0.00,0.14
358209,2021-10-05,Teton,Wyoming,4956,13.00,56039,WY,County,23464,21121.72,55.40,55.40,21241.05,23,1,3.00,9.79,0.00,0.07
358210,2021-10-06,Teton,Wyoming,4960,13.00,56039,WY,County,23464,21138.77,55.40,55.40,21241.05,23,1,4.00,7.79,0.00,0.07
358211,2021-10-07,Teton,Wyoming,4973,13.00,56039,WY,County,23464,21194.17,55.40,55.40,21241.05,23,1,13.00,7.36,0.00,0.07


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-10-08') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
432743,2021-10-08,Imperial,California,35225,759.00,6025,CA,County,181215,19438.24,418.84,418.84,19438.24,1,3,59.00,54.50,0.00,0.36
3117,2021-10-08,Los Angeles,California,1469338,26284.00,6037,CA,County,10039107,14636.14,261.82,261.82,14636.14,2,12,1184.00,1227.93,30.00,22.29
214843,2021-10-08,San Bernardino,California,354330,5557.00,6071,CA,County,2180085,16253.04,254.90,254.90,16253.04,3,5,430.00,377.36,2.00,1.21
95765,2021-10-08,Stanislaus,California,84010,1301.00,6099,CA,County,550660,15256.24,236.26,236.26,15256.24,4,7,236.00,180.93,4.00,3.21
76115,2021-10-08,San Joaquin,California,100830,1681.00,6077,CA,County,762148,13229.71,220.56,220.56,13229.71,5,18,188.00,177.07,13.00,2.86
759892,2021-10-08,Inyo,California,1837,39.00,6027,CA,County,18039,10183.49,216.20,216.20,10183.49,6,31,2.00,10.86,0.00,0.00
688281,2021-10-08,Merced,California,41599,570.00,6047,CA,County,277680,14980.91,205.27,205.27,14980.91,7,8,144.00,105.64,0.00,1.29
44217,2021-10-08,Riverside,California,366901,5063.00,6065,CA,County,2470546,14851.01,204.93,204.93,14851.01,8,10,437.00,488.79,8.00,8.00
43055,2021-10-08,Fresno,California,142025,2018.00,6019,CA,County,999101,14215.28,201.98,201.98,14215.28,9,13,434.00,331.29,10.00,6.50
922460,2021-10-08,Kings,California,31787,308.00,6031,CA,County,152940,20783.97,201.39,201.39,20783.97,10,2,109.00,97.71,0.00,1.07


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1619764,2021-10-08,Lassen,California,7318,39.00,6035,CA,County,30573,23936.15,127.56,127.56,23936.15,28,1,19.00,16.36,0.00,0.21
922460,2021-10-08,Kings,California,31787,308.00,6031,CA,County,152940,20783.97,201.39,201.39,20783.97,10,2,109.00,97.71,0.00,1.07
432743,2021-10-08,Imperial,California,35225,759.00,6025,CA,County,181215,19438.24,418.84,418.84,19438.24,1,3,59.00,54.50,0.00,0.36
118842,2021-10-08,Tulare,California,77528,932.00,6107,CA,County,466195,16629.95,199.92,199.92,16629.95,11,4,200.00,241.86,6.00,2.43
214843,2021-10-08,San Bernardino,California,354330,5557.00,6071,CA,County,2180085,16253.04,254.90,254.90,16253.04,3,5,430.00,377.36,2.00,1.21
270369,2021-10-08,Kern,California,141890,1582.00,6029,CA,County,900202,15762.02,175.74,175.74,15762.02,14,6,427.00,354.14,9.00,3.57
95765,2021-10-08,Stanislaus,California,84010,1301.00,6099,CA,County,550660,15256.24,236.26,236.26,15256.24,4,7,236.00,180.93,4.00,3.21
688281,2021-10-08,Merced,California,41599,570.00,6047,CA,County,277680,14980.91,205.27,205.27,14980.91,7,8,144.00,105.64,0.00,1.29
985849,2021-10-08,Glenn,California,4228,39.00,6021,CA,County,28393,14890.99,137.36,137.36,14890.99,24,9,7.00,10.93,1.00,0.29
44217,2021-10-08,Riverside,California,366901,5063.00,6065,CA,County,2470546,14851.01,204.93,204.93,14851.01,8,10,437.00,488.79,8.00,8.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
432743,2021-10-08,Imperial,California,35225,759.00,6025,CA,County,181215,19438.24,418.84,418.84,19438.24,1,3,59.00,54.50,0.00,0.36
3117,2021-10-08,Los Angeles,California,1469338,26284.00,6037,CA,County,10039107,14636.14,261.82,261.82,14636.14,2,12,1184.00,1227.93,30.00,22.29
214843,2021-10-08,San Bernardino,California,354330,5557.00,6071,CA,County,2180085,16253.04,254.90,254.90,16253.04,3,5,430.00,377.36,2.00,1.21
95765,2021-10-08,Stanislaus,California,84010,1301.00,6099,CA,County,550660,15256.24,236.26,236.26,15256.24,4,7,236.00,180.93,4.00,3.21
76115,2021-10-08,San Joaquin,California,100830,1681.00,6077,CA,County,762148,13229.71,220.56,220.56,13229.71,5,18,188.00,177.07,13.00,2.86
759892,2021-10-08,Inyo,California,1837,39.00,6027,CA,County,18039,10183.49,216.20,216.20,10183.49,6,31,2.00,10.86,0.00,0.00
688281,2021-10-08,Merced,California,41599,570.00,6047,CA,County,277680,14980.91,205.27,205.27,14980.91,7,8,144.00,105.64,0.00,1.29
44217,2021-10-08,Riverside,California,366901,5063.00,6065,CA,County,2470546,14851.01,204.93,204.93,14851.01,8,10,437.00,488.79,8.00,8.00
43055,2021-10-08,Fresno,California,142025,2018.00,6019,CA,County,999101,14215.28,201.98,201.98,14215.28,9,13,434.00,331.29,10.00,6.50
922460,2021-10-08,Kings,California,31787,308.00,6031,CA,County,152940,20783.97,201.39,201.39,20783.97,10,2,109.00,97.71,0.00,1.07


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'state_case', 'case_day', '14day_case_avg']]
top10_ca_case_df

,date,county,cases,current_crate,state_case,case_day,14day_case_avg
0,03/20/20,Imperial,4,2.21,3,4.00,98.00
1,03/21/20,Imperial,4,2.21,3,0.00,98.00
2,03/22/20,Imperial,4,2.21,3,0.00,77.00
3,03/23/20,Imperial,4,2.21,3,0.00,72.14
4,03/24/20,Imperial,9,4.97,3,5.00,62.29
...,...,...,...,...,...,...,...
5630,10/04/21,Lassen,7236,23667.94,1,0.00,23.00
5631,10/05/21,Lassen,7288,23838.03,1,52.00,22.21
5632,10/06/21,Lassen,7299,23874.01,1,11.00,18.86
5633,10/07/21,Lassen,7299,23874.01,1,0.00,17.57


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'state_death', 'death_day', '14day_death_avg']]
top10_ca_death_df

,date,county,deaths,current_drate,state_death,death_day,14day_death_avg
0,03/20/20,Imperial,0.00,0.00,1,0.00,0.57
1,03/21/20,Imperial,0.00,0.00,1,0.00,0.57
2,03/22/20,Imperial,0.00,0.00,1,0.00,0.50
3,03/23/20,Imperial,0.00,0.00,1,0.00,0.50
4,03/24/20,Imperial,0.00,0.00,1,0.00,0.50
...,...,...,...,...,...,...,...
5763,10/04/21,Kings,300.00,196.16,10,-1.00,1.14
5764,10/05/21,Kings,303.00,198.12,10,3.00,1.07
5765,10/06/21,Kings,303.00,198.12,10,0.00,0.86
5766,10/07/21,Kings,308.00,201.39,10,5.00,1.14


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)